Read features.csv

In [10]:
import pandas as pd
from pathlib import Path

split_name = "train_test"
features_path = Path("resources") / split_name / "features.csv"

features_df = pd.read_csv(features_path)
features_df.head()

name   term freq  ...     hurst stability  lumpiness
0  loop_seattle  short   5T  ...  0.936951  0.458372   0.613632
1  loop_seattle  short    D  ...  0.611051  0.170957   0.339067
2  loop_seattle  short    H  ...  0.689547  0.211964   0.454267
3       m_dense  short    D  ...  0.744518  0.353943   0.367117
4       m_dense  short    H  ...  0.588340  0.113609   0.155365

[5 rows x 11 columns]

Initialize a col for CRPS scores

In [11]:
import numpy as np

features_df["true_crps"] = np.nan
features_df.head()

name   term freq  ...  stability lumpiness  true_crps
0  loop_seattle  short   5T  ...   0.458372  0.613632        NaN
1  loop_seattle  short    D  ...   0.170957  0.339067        NaN
2  loop_seattle  short    H  ...   0.211964  0.454267        NaN
3       m_dense  short    D  ...   0.353943  0.367117        NaN
4       m_dense  short    H  ...   0.113609  0.155365        NaN

[5 rows x 12 columns]

Read CRPS scores

In [12]:
model_name = "chronos_bolt_base"
scores_path = Path("results") / model_name / "all_results.csv"
scores_df = pd.read_csv(scores_path)
scores_df.head()

dataset              model  ...        domain  num_variates
0           ett1/W/short  chronos_bolt_base  ...        Energy             7
1  bitbrains_rnd/H/short  chronos_bolt_base  ...  Web/CloudOps             2
2     m4_monthly/M/short  chronos_bolt_base  ...      Econ/Fin             1
3        solar/10T/short  chronos_bolt_base  ...        Energy             1
4       solar/10T/medium  chronos_bolt_base  ...        Energy             1

[5 rows x 15 columns]

Combine scores df with dataset info df

In [13]:
for i, score_row in scores_df.iterrows():
    name, freq, term = score_row["dataset"].split("/")
    name = name.lower()
    freq = freq.split("-")[0]

    mask = (
        (features_df["name"].str.lower() == name)
        & (features_df["freq"] == freq)
        & (features_df["term"] == term)
    )

    features_df.loc[mask, "true_crps"] = score_row[
        "eval_metrics/mean_weighted_sum_quantile_loss"
    ]


features_df.head()
features_df.to_csv(features_path, index=False)